In [1]:
import sys, os
parent_dir = os.getcwd()
path = os.path.dirname(parent_dir)
sys.path.append(path)

from gym import spaces
from gym_homer.envs.test_env import HomerEnv
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(path+"/test_env_data.csv")
data.loc[:,'time']=data.apply(lambda x: np.array([x.time_x, x.time_y], dtype=np.float32), axis=1)
data.loc[:,'month']=data.apply(lambda x: np.array([x.month_x, x.month_y], dtype=np.float32), axis=1)
data.loc[:,'region']=data.apply(lambda x: np.array([x.region_1, x.region_2, x.region_3], dtype=np.int8), axis=1)
data.loc[:,'weekend']=data.apply(lambda x: np.array(x.weekend, dtype=np.int8), axis=1)

In [3]:
data

,time_x,time_y,weekend,month_x,month_y,region_1,region_2,region_3,solar,home,import_tariff,export_tariff,time,month,region
0,0.000,1.000,0,0.0,1.0,0,0,0,0,1,0.05,0.01,"[0.0, 1.0]","[0.0, 1.0]","[0, 0, 0]"
1,0.500,0.866,0,0.0,1.0,0,0,0,1,1,0.05,0.01,"[0.5, 0.866]","[0.0, 1.0]","[0, 0, 0]"
2,0.866,0.500,0,0.0,1.0,0,0,0,1,1,0.05,0.01,"[0.866, 0.5]","[0.0, 1.0]","[0, 0, 0]"
3,1.000,0.000,0,0.0,1.0,0,0,0,0,1,0.05,0.01,"[1.0, 0.0]","[0.0, 1.0]","[0, 0, 0]"
4,0.866,-0.500,0,0.0,1.0,0,0,0,0,1,0.05,0.01,"[0.866, -0.5]","[0.0, 1.0]","[0, 0, 0]"
5,0.500,-0.866,0,0.0,1.0,0,0,0,0,1,0.05,0.01,"[0.5, -0.866]","[0.0, 1.0]","[0, 0, 0]"
6,0.000,-1.000,0,0.0,1.0,0,0,0,0,1,0.05,0.01,"[0.0, -1.0]","[0.0, 1.0]","[0, 0, 0]"
7,-0.500,-0.866,0,0.0,1.0,0,0,0,0,1,0.05,1.00,"[-0.5, -0.866]","[0.0, 1.0]","[0, 0, 0]"
8,-0.866,-0.500,0,0.0,1.0,0,0,0,0,1,0.05,1.00,"[-0.866, -0.5]","[0.0, 1.0]","[0, 0, 0]"
9,-1.000,-0.000,0,0.0,1.0,0,0,0,0,1,0.05,0.01,"[-1.0, -0.0]","[0.0, 1.0]","[0, 0, 0]"


In [4]:
data.solar.sum() * 1 - (data.home.sum() * 0.05)

1.4

In [23]:
import gym, torch, numpy as np, torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import tianshou as ts
from gym import spaces, wrappers

In [24]:
#task = 'HomerEnv-v0'
lr, epoch, batch_size = 1e-3, 10, 64
train_num, test_num = 10, 100
gamma, n_step, target_freq = 0.9, 3, 320
buffer_size = 20000
eps_train, eps_test = 0.1, 0.05
step_per_epoch, step_per_collect = 10000, 10
logger = ts.utils.TensorboardLogger(SummaryWriter('log/dqn'))  # TensorBoard is supported!
# For other loggers: https://tianshou.readthedocs.io/en/master/tutorials/logger.html

In [34]:
# you can also try with SubprocVectorEnv
train_envs = ts.env.DummyVectorEnv([gym.make('gym_homer/HomerEnv-v0', data=data) for _ in range(train_num)])
test_envs = ts.env.DummyVectorEnv([gym.make('gym_homer/HomerEnv-v0', data=data) for _ in range(test_num)])

TypeError: 'OrderEnforcing' object is not callable

In [26]:
len(env.observation_space.items())

11

In [27]:
from tianshou.utils.net.common import Net
# you can define other net by following the API:
# https://tianshou.readthedocs.io/en/master/tutorials/dqn.html#build-the-network
env = gym.make('gym_homer/HomerEnv-v0', data=data)
wrapped_env = wrappers.FlattenObservation(env)
state_shape = wrapped_env.observation_space.shape or wrapped_env.observation_space.n
action_shape = wrapped_env.action_space.shape or wrapped_env.action_space.n
net = Net(state_shape=state_shape, action_shape=action_shape, hidden_sizes=[128, 128, 128])
optim = torch.optim.Adam(net.parameters(), lr=lr)

In [28]:
policy = ts.policy.DQNPolicy(net, optim, gamma, n_step, target_update_freq=target_freq)
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(buffer_size, train_num), exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)  # because DQN uses epsilon-greedy method

NameError: name 'train_envs' is not defined

In [35]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector, epoch, step_per_epoch, step_per_collect,
    test_num, batch_size, update_per_step=1 / step_per_collect,
    train_fn=lambda epoch, env_step: policy.set_eps(eps_train),
    test_fn=lambda epoch, env_step: policy.set_eps(eps_test),
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold,
    logger=logger)
print(f'Finished training! Use {result["duration"]}')

NameError: name 'train_collector' is not defined

In [36]:
policy.eval()
policy.set_eps(eps_test)
collector = ts.data.Collector(policy, env, exploration_noise=True)
collector.collect(n_episode=1, render=1 / 35)

/Users/mbtalexander/Berkeley/w210/home-energy-optimizer/ts/lib/python3.9/site-packages/tianshou/data/collector.py:66: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

SyntaxError: invalid syntax (1805115089.py, line 1)